In [1]:
!nvidia-smi

Mon Jun 28 16:17:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install emoji -q
!pip install datasets -q
!pip install transformers -q
!pip install sentencepiece -q
!pip install rouge_score -q

     |████████████████████████████████| 133kB 8.2MB/s 
     |████████████████████████████████| 245kB 7.9MB/s 
     |████████████████████████████████| 245kB 45.1MB/s 
     |████████████████████████████████| 122kB 54.8MB/s 
     |████████████████████████████████| 2.5MB 7.0MB/s 
     |████████████████████████████████| 901kB 45.3MB/s 
     |████████████████████████████████| 3.3MB 44.4MB/s 
     |████████████████████████████████| 1.2MB 8.0MB/s 


## Importing the Required Libraries

In [3]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import datasets
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, average_precision_score, precision_score
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
from typing import Optional
from dataclasses import dataclass, field
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer, default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments
                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [4]:
# Setting seed
set_seed(321)
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)
device = 'cuda'
nltk.download('punkt')
metric = datasets.load_metric('rouge')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Loading the Dataset

In [8]:
dataset_path = '/content/drive/MyDrive/EMNLP_folder_4/dataset/'

In [9]:
df_train = pd.read_csv(dataset_path + 'train_tfidf.csv')
df_val = pd.read_csv(dataset_path + 'val_tfidf.csv')
df_test = pd.read_csv(dataset_path + 'final_test.csv')

In [10]:
df_test.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,arg_id,key_point_id,arg,key_point,topic,stance,kp_arg_topic,dependency_features,noun_features,encoded_dependency_features,encoded_noun_features,encoded_noun_features_len,encoded_dependency_features_len,tf_idf_bigram_features,tf_idf_bigram_len
0,0,0,0,arg_0_0,kp_0_0,Routine child vaccinations isn't mandatory sin...,"Routine child vaccinations, or their side effe...",Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe...","['amod', 'compound', 'nsubj', 'punct', 'cc', '...","['NOUN', 'NOUN', 'AUX', 'NOUN', 'NOUN', 'NOUN'...","[1, 23, 2, 18, 16, 12, 23, 11, 18, 3, 36, 23, ...","[1, 1, 4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 4, 4, 4, ...",28,28,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",408
1,1,1,1,arg_0_0,kp_0_1,Routine child vaccinations isn't mandatory sin...,Mandatory vaccination contradicts basic rights,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights...,"['amod', 'nsubj', 'nsubj', 'amod', 'amod', 'co...","['NOUN', 'VERB', 'AUX', 'NOUN', 'NOUN', 'NOUN'...","[1, 2, 2, 1, 1, 23, 2, 22, 26, 17, 27, 2, 9, 2...","[1, 2, 4, 1, 1, 1, 4, 2, 4, 4, 2, 2, 2, 2, 4, ...",22,22,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",408
2,2,2,2,arg_0_0,kp_0_2,Routine child vaccinations isn't mandatory sin...,The parents and not the state should decide,Routine child vaccinations should be mandatory,-1,The parents and not the state should decideRou...,"['det', 'nsubj', 'cc', 'neg', 'det', 'conj', '...","['NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN...","[6, 2, 16, 26, 6, 11, 9, 1, 23, 2, 3, 26, 17, ...","[1, 2, 1, 1, 1, 1, 4, 1, 1, 4, 4, 4, 4, 2, 2, ...",25,25,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",408
3,3,3,3,arg_0_0,kp_0_3,Routine child vaccinations isn't mandatory sin...,Routine child vaccinations are not necessary t...,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary t...,"['amod', 'compound', 'nsubj', 'ccomp', 'neg', ...","['NOUN', 'NOUN', 'AUX', 'AUX', 'AUX', 'AUX', '...","[1, 23, 2, 22, 26, 17, 9, 10, 4, 23, 23, 2, 22...","[1, 1, 4, 4, 4, 4, 2, 5, 2, 1, 1, 4, 2, 4, 4, ...",27,27,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",408
4,4,4,4,arg_0_1,kp_0_0,Routine child vaccinations should not be manda...,"Routine child vaccinations, or their side effe...",Routine child vaccinations should be mandatory,-1,"Routine child vaccinations, or their side effe...","['amod', 'compound', 'nsubj', 'punct', 'cc', '...","['NOUN', 'NOUN', 'AUX', 'NOUN', 'NOUN', 'NOUN'...","[1, 23, 2, 18, 16, 12, 23, 11, 18, 3, 36, 23, ...","[1, 1, 4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 4, 4, 4, ...",34,34,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",408


In [11]:
config = PegasusConfig.from_pretrained('google/pegasus-xsum')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

In [12]:
def gen_arg_topic(df):

    arg_topic = []

    for i in range(len(df)):
        arg_topic.append(df['arg'][i] + df['topic'][i])
    
    df['arg_topic'] = arg_topic
    return df

df_train = gen_arg_topic(df_train)
df_val = gen_arg_topic(df_val)
df_test = gen_arg_topic(df_test)

In [13]:
df_train = pd.concat([df_train,df_val])

df_train.to_csv(dataset_path + 'gen_train.csv')
df_val.to_csv(dataset_path + 'gen_val.csv')
df_test.to_csv(dataset_path + 'gen_test.csv')

In [14]:
# Get the column names for input/target.
dataset_columns = ('arg_topic', 'key_point')
arg_topic_column = dataset_columns[0]
key_point_column = dataset_columns[1]

# Temporarily set max_target_length for training.
max_source_length = 64
max_target_length = 64
padding = "max_length" 

device='cuda'

train_dataset = datasets.load_dataset('csv', data_files = dataset_path + 'gen_train.csv')['train']
val_dataset = datasets.load_dataset('csv', data_files = dataset_path + 'gen_val.csv')['train']
test_dataset = datasets.load_dataset('csv', data_files = dataset_path + 'gen_test.csv')['train']

Using custom data configuration default-6600cb8ae2993039


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6600cb8ae2993039/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Using custom data configuration default-c83fdb72faaea32c


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c83fdb72faaea32c/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Using custom data configuration default-3ce4ea71e76f51f7


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3ce4ea71e76f51f7/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [15]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1', 'arg_id', 'key_point_id', 'label', 'arg', 'key_point', 'topic', 'stance', 't5_key_point', 'arg_token_len', 'kp_token_len', 't5_kp_token_len', 'topic_token_len', 'kp_arg_topic', 'dependency_features', 'noun_features', 'encoded_dependency_features', 'encoded_noun_features', 'encoded_noun_features_len', 'encoded_dependency_features_len', 'tf_idf_features', 'tf_idf_len', 'tf_idf_bigram_features', 'tf_idf_bigram_len', 'tf_idf_trigram_features', 'tf_idf_trigram_len', 'arg_topic'],
    num_rows: 24093
})

In [16]:
val_dataset

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1', 'arg_id', 'key_point_id', 'label', 'arg', 'key_point', 'topic', 'stance', 't5_key_point', 'arg_token_len', 'kp_token_len', 't5_kp_token_len', 'topic_token_len', 'kp_arg_topic', 'dependency_features', 'noun_features', 'encoded_dependency_features', 'encoded_noun_features', 'encoded_noun_features_len', 'encoded_dependency_features_len', 'tf_idf_features', 'tf_idf_len', 'tf_idf_bigram_features', 'tf_idf_bigram_len', 'tf_idf_trigram_features', 'tf_idf_trigram_len', 'arg_topic'],
    num_rows: 3458
})

In [17]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'arg_id', 'key_point_id', 'arg', 'key_point', 'topic', 'stance', 'kp_arg_topic', 'dependency_features', 'noun_features', 'encoded_dependency_features', 'encoded_noun_features', 'encoded_noun_features_len', 'encoded_dependency_features_len', 'tf_idf_bigram_features', 'tf_idf_bigram_len', 'arg_topic'],
    num_rows: 3923
})

In [18]:
data = {'train': train_dataset, 'validation': val_dataset, 'test': test_dataset}

column_names = data["train"].column_names

prefix=""

In [19]:
column_names

['Unnamed: 0',
 'Unnamed: 0.1',
 'Unnamed: 0.1.1',
 'Unnamed: 0.1.1.1',
 'Unnamed: 0.1.1.1.1',
 'Unnamed: 0.1.1.1.1.1',
 'Unnamed: 0.1.1.1.1.1.1',
 'arg_id',
 'key_point_id',
 'label',
 'arg',
 'key_point',
 'topic',
 'stance',
 't5_key_point',
 'arg_token_len',
 'kp_token_len',
 't5_kp_token_len',
 'topic_token_len',
 'kp_arg_topic',
 'dependency_features',
 'noun_features',
 'encoded_dependency_features',
 'encoded_noun_features',
 'encoded_noun_features_len',
 'encoded_dependency_features_len',
 'tf_idf_features',
 'tf_idf_len',
 'tf_idf_bigram_features',
 'tf_idf_bigram_len',
 'tf_idf_trigram_features',
 'tf_idf_trigram_len',
 'arg_topic']

In [20]:
test_column_names = data["test"].column_names
test_column_names

['Unnamed: 0',
 'Unnamed: 0.1',
 'Unnamed: 0.1.1',
 'Unnamed: 0.1.1.1',
 'arg_id',
 'key_point_id',
 'arg',
 'key_point',
 'topic',
 'stance',
 'kp_arg_topic',
 'dependency_features',
 'noun_features',
 'encoded_dependency_features',
 'encoded_noun_features',
 'encoded_noun_features_len',
 'encoded_dependency_features_len',
 'tf_idf_bigram_features',
 'tf_idf_bigram_len',
 'arg_topic']

In [21]:
def preprocess_function(examples):
    inputs = examples[arg_topic_column]
    targets = examples[key_point_column]
    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

eval_dataset = val_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

test_dataset = test_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=test_column_names,
            load_from_cache_file=False,
        )

In [23]:
label_pad_token_id = -100 
data_collator = default_data_collator

metric_name = "rouge"
metric = datasets.load_metric(metric_name)

In [24]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [25]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    if metric_name == "rouge":
        result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        # Extract a few results from ROUGE
        result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    else:
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [26]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/EMNLP_folder_4/headline_model',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/EMNLP_folder_4/headline_model_logs',            # directory for storing logs
    save_steps = 4518, #in order to store the last model at the end of 3 epochs
    predict_with_generate=True
)

# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [27]:
model.cuda();

In [28]:
checkpoint = ''
train_result = trainer.train(resume_from_checkpoint=None)
trainer.save_model()

***** Running training *****
  Num examples = 24093
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4518


Step,Training Loss
500,2.640900
1000,0.596900
1500,0.343700
2000,0.290000
2500,0.264700
3000,0.246000
3500,0.228000
4000,0.222700
4500,0.214600


Saving model checkpoint to /content/drive/MyDrive/EMNLP_folder_4/headline_model/checkpoint-4518
Configuration saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/checkpoint-4518/config.json
Model weights saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/checkpoint-4518/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/checkpoint-4518/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/checkpoint-4518/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/EMNLP_folder_4/headline_model
Configuration saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/config.json
Model weights saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/EMNLP_folder_4/headline_model/tokenizer_conf

In [29]:
train_result

TrainOutput(global_step=4518, training_loss=0.5594840211222372, metrics={'train_runtime': 5804.0856, 'train_samples_per_second': 12.453, 'train_steps_per_second': 0.778, 'total_flos': 1.581344654819328e+16, 'train_loss': 0.5594840211222372, 'epoch': 3.0})

In [30]:
test_results = trainer.predict(
      test_dataset,
      metric_key_prefix="test",
      max_length=max_target_length,
      num_beams=6
)

***** Running Prediction *****
  Num examples = 3923
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [31]:
test_results

PredictionOutput(predictions=array([[    0, 10665,   138, ...,     0,     0,     0],
       [    0, 10665,   138, ...,     0,     0,     0],
       [    0, 10665,   138, ...,     0,     0,     0],
       ...,
       [    0,  2184,   246, ...,     0,     0,     0],
       [    0,  2184,   246, ...,     0,     0,     0],
       [    0,  2184,   246, ...,     0,     0,     0]]), label_ids=array([[40258,   667, 28358, ...,  -100,  -100,  -100],
       [68021, 19138, 61700, ...,  -100,  -100,  -100],
       [  139,  1119,   111, ...,  -100,  -100,  -100],
       ...,
       [  139,   787,   148, ...,  -100,  -100,  -100],
       [  139,   787,   148, ...,  -100,  -100,  -100],
       [  139,   787,   148, ...,  -100,  -100,  -100]]), metrics={'test_loss': 5.64322566986084, 'test_rouge1': 20.0222, 'test_rouge2': 4.8533, 'test_rougeL': 18.0097, 'test_rougeLsum': 18.0055, 'test_gen_len': 11.8228, 'test_runtime': 378.6764, 'test_samples_per_second': 10.36, 'test_steps_per_second': 0.65})

## Saving the predictions of model

In [32]:
if trainer.is_world_process_zero():
  if training_args.predict_with_generate:
      test_preds = tokenizer.batch_decode(
          test_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
      )
      test_preds = [pred.strip() for pred in test_preds]
      output_test_preds_file = os.path.join('/content/drive/MyDrive/EMNLP_folder_4/gen_key_points_predictions/', "test_set_generations.txt")
      with open(output_test_preds_file, "w") as writer:
          writer.write("\n".join(test_preds))